In [ ]:
def define_model_dropout(dropout_rate = 0):
  model = Sequential()
  model.add(Conv2D(32, (3,3), padding='same', input_shape=(300, 300, 4)))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(dropout_rate))

  model.add(Conv2D(64, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(dropout_rate))

  model.add(Conv2D(128, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(dropout_rate))
  
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(dropout_rate))

  model.add(Flatten())
  model.add(Dense(1, activation='softmax'))
  return(model)

In [ ]:
# osaliselt praktikumi ja kodutöö materjalidest

from sklearn.model_selection import KFold
dropout_rates = [0.5, 0.6, 0.75, 0.85, 0.9, 0.99]

val_fold_acc = np.zeros(len(dropout_rates))

for i, dropout_rate in enumerate(dropout_rates):
  print(f'Validation loss for dropout rate = {dropout_rate}...')

  # 4-fold cross validation
  kf = KFold(n_splits=4, shuffle=True, random_state=111)

  fold_acc = []
  for train_index, val_index in kf.split(X_train_norm):

    # split data into train_X, train_y and val_X, val_y depending on the fold:
    train_X = X_train_norm[train_index]
    y_train = np.array(y_train)
    train_Y = y_train[train_index]
    val_X = X_train_norm[val_index]
    val_Y = y_train[val_index]

    # train the neural network with dropout_rate
    model = define_model_dropout(dropout_rate)

    # compile the model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=[tf.keras.metrics.BinaryAccuracy(),
                      tf.keras.metrics.FalseNegatives()], 
                      optimizer=optimizers.Adam(learning_rate=0.001))

    # fit the neural network on training data
    history = model.fit(np.array(train_X), np.array(train_Y), batch_size=10, epochs=15)

    # calculate accuracy for this fold and store it in fold_acc
    print(model.evaluate(val_X, val_Y))
    #acc, loss = model.evaluate(val_X, val_Y)
    #fold_acc.append(acc)
    

  #print(f'Average validation accuracy for {dropout_rate} is {np.mean(fold_acc)}')
  #val_fold_acc[i] = np.mean(fold_acc)